In [12]:

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import time

from six.moves import xrange  # pylint: disable=redefined-builtin

#from tensorflow.contrib.learn.python.learn.datasets import base
from tensorflow.python.framework import dtypes
from sklearn.model_selection import train_test_split

import random
import pandas as pd
import tensorflow as tf

import datetime
from tensorflow.python.framework import dtypes
import numpy as np
import sklearn

RANDOM_SEED = int(time.time())
tf.set_random_seed(RANDOM_SEED)



### reading CSV row data

In [13]:
trainDF = pd.read_csv('Data/adult_data_train.csv', na_values=['?'], delimiter='\t', dtype=np.float64)
trainData = trainDF.drop(columns=['target_cat'])
trainLabel = trainDF['target_cat'].astype(np.int32)


In [14]:
testDF = pd.read_csv('Data/adult_data_test.csv', na_values=['?'], delimiter='\t', dtype=np.float64)
testData = testDF.drop(columns=['target_cat'])
testLabel = testDF['target_cat'].astype(np.int32)


In [15]:
def make_one_hot(labels):
    num_labels = len(np.unique(labels))
    return np.eye(num_labels)[labels]  # One liner trick!

trainLabel = make_one_hot(trainLabel)
testLabel  = make_one_hot(testLabel)

trainData, validationData, trainLabel, validationLabel = train_test_split(trainData, trainLabel, test_size=1000, random_state=RANDOM_SEED)




In [16]:
print (trainData.shape)
print (trainLabel.shape)
print (validationData.shape)
print (validationLabel.shape)
print (testData.shape)
print (testLabel.shape)


(31561, 107)
(31561, 2)
(1000, 107)
(1000, 2)
(16281, 107)
(16281, 2)


## aux functions

In [17]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

def variable_summaries(var):
    """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)


## main parameters


In [18]:
# Parameters
display_step               = 100
batch_size                 = 128
num_steps                  = 10000



# Network Parameters
n_hidden_1 = 150 # 1st layer number of neurons
n_hidden_2 = 100  # 2nd layer number of neurons
n_hidden_3 = 50  # 3rd layer number of neurons
n_hidden_4 = 25  # 3rd layer number of neurons

num_input = 107 # income data input (vec size: 107)
num_classes = 2 # income total classes (0 - 1 )


### AUX functions

In [19]:
def calcTestMetrics(data, labels):
    prediction=tf.argmax(logits,1)
    best = sess.run([prediction],feed_dict={X: data, Y: labels})
    predictions = best[0]
    trueValues = []
    for i in labels:
        if  np.all(i == [0,1]):
            trueValues.append(1)
        else:
            trueValues.append(0)

    return sklearn.metrics.classification_report(digits=2, y_true=trueValues, y_pred=predictions)


## Four hidden layers

In [21]:
graph = tf.Graph()
with graph.as_default():
    with tf.name_scope("input_layers"):
    # tf Graph input
        X                          = tf.placeholder("float64", [None, num_input], name="trainData")
        Y                          = tf.placeholder("float64", [None, num_classes], name="trainLabel")

        validation_dataset         = tf.constant(validationData)
        testing_dataset            = tf.constant(testData)

    with tf.name_scope('variables'):
        initial_learning_rate      = tf.placeholder(tf.float32, name="initial_learning_rate")
    
        beta_regul                 = tf.placeholder(tf.float64, name="beta_regulation")
        global_step                = tf.Variable(0, name="global_step")
        keep_prob                  = tf.placeholder(tf.float64, name="keep_prob")
    
        variable_summaries(beta_regul)
        variable_summaries(keep_prob)

    


    # Store layers weight & bias
    with tf.name_scope("wights"):
        weights = {
            'h1': tf.Variable(tf.truncated_normal([num_input, n_hidden_1], mean=0,  stddev=3.0, dtype=tf.float64),name="hidden_layer_1"),
            'h2': tf.Variable( tf.truncated_normal([n_hidden_1, n_hidden_2], mean=0,  stddev=3.0, dtype=tf.float64), name="hidden_layer_2"),
            'h3': tf.Variable( tf.truncated_normal([n_hidden_2, n_hidden_3], mean=0,  stddev=3.0, dtype=tf.float64), name="hidden_layer_3"),
            'h4': tf.Variable( tf.truncated_normal([n_hidden_3, n_hidden_4], mean=0,  stddev=3.0, dtype=tf.float64), name="hidden_layer_4"),
            'out': tf.Variable( tf.truncated_normal([n_hidden_4, num_classes], mean=0, stddev=3.0, dtype=tf.float64), name="output_layer" )
            }
        variable_summaries(weights["h1"])
        variable_summaries(weights["h2"])
        variable_summaries(weights["h3"])
        variable_summaries(weights["h4"])
        variable_summaries(weights["out"])
        
    with tf.name_scope("biases"):
        biases = {
            'b1': tf.Variable(  tf.zeros([n_hidden_1], dtype=tf.float64), dtype=tf.float64, name="hidden_layer_1"),
            'b2': tf.Variable( tf.zeros([n_hidden_2], dtype=tf.float64), dtype=tf.float64, name="hidden_layer_2"),
            'b3': tf.Variable( tf.zeros([n_hidden_3], dtype=tf.float64), dtype=tf.float64, name="hidden_layer_3"),
            'b4': tf.Variable( tf.zeros([n_hidden_4], dtype=tf.float64), dtype=tf.float64, name="hidden_layer_4"),    
            'out': tf.Variable( tf.zeros([num_classes], dtype=tf.float64), dtype=tf.float64, name="output_layer")
            }
        variable_summaries(biases["b1"])
        variable_summaries(biases["b2"])
        variable_summaries(biases["b3"])
        variable_summaries(biases["b4"])
        variable_summaries(biases["out"])


    # Create model
    # Training computation.
    with tf.name_scope("model"):
        preactivation_h1 = tf.add(tf.matmul(X, weights['h1']) , biases['b1'], name="h1_preactivation")
        training_h1 = tf.nn.sigmoid (preactivation_h1, name="h1_output")
        h1_dropout = tf.nn.dropout(training_h1, keep_prob=keep_prob, name="h1_dropout")
        tf.summary.histogram('h1_pre_activations', preactivation_h1)
        tf.summary.histogram('h1_output', training_h1)
        tf.summary.histogram('h1_dropout', h1_dropout)

        preactivation_h2 = tf.add(tf.matmul(h1_dropout, weights['h2']) , biases['b2'])
        training_h2 = tf.nn.sigmoid(preactivation_h2, name="h2_output")
        h2_dropout = tf.nn.dropout(training_h2, keep_prob=keep_prob, name="h2_dropout")
        tf.summary.histogram('h2_pre_activations', preactivation_h2)
        tf.summary.histogram('h2_output', training_h2)
        tf.summary.histogram('h2_dropout', h2_dropout)

        preactivation_h3 = tf.add(tf.matmul(h2_dropout, weights['h3']), biases['b3'])
        training_h3 = tf.nn.sigmoid(preactivation_h3, name="h3_output")
        h3_dropout = tf.nn.dropout(training_h3, keep_prob=keep_prob, name="h3_dropout")
        tf.summary.histogram('h3_pre_activations', preactivation_h3)
        tf.summary.histogram('h3_output', training_h3)
        tf.summary.histogram('h3_dropout', h3_dropout)
                                    
        preactivation_h4 = tf.add(tf.matmul(h3_dropout, weights['h4']), biases['b4'])
        training_h4 = tf.nn.sigmoid(preactivation_h4, name="h4_output")
        h4_dropout = tf.nn.dropout(training_h4, keep_prob=keep_prob, name="h4_dropout")
        tf.summary.histogram('h4_pre_activations', preactivation_h4)
        tf.summary.histogram('h4_output', training_h4)
        tf.summary.histogram('h4_dropout', h4_dropout)

        logits = tf.add(tf.matmul(h4_dropout, weights['out']), biases['out'], name="logits")
        tf.summary.histogram('output_pre_activations', logits)

    # Define loss and optimizerimizer

    with tf.name_scope("loss"):
        loss = tf.reduce_mean( tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=Y) )  + \
                   beta_regul * (tf.nn.l2_loss(weights['h1']) + tf.nn.l2_loss(weights['h2']) + \
                                 tf.nn.l2_loss(weights['h3']) + tf.nn.l2_loss(weights['h4']) + \
                                 tf.nn.l2_loss(weights['out']) )    
        tf.summary.scalar('loss', loss)


    # Optimizer.
    with tf.name_scope("Optimizer"):
        learning_rate = tf.train.exponential_decay(initial_learning_rate, global_step, 1000, 0.10, staircase=True, name="lr_decay")
        tf.summary.scalar('learning_rate', learning_rate)
        optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate, name="RMSProp").minimize(loss, name="minimizer")

    # Predictions for the training, validation, and test data.
    
    train_prediction      = tf.nn.softmax(logits, name="train_pred")

    with tf.name_scope("validation"): 
        validation_h1         = tf.nn.sigmoid(tf.matmul(validationData, weights['h1']) + biases['b1'])
        validation_h2         = tf.nn.sigmoid(tf.matmul(validation_h1, weights['h2']) + biases['b2'])
        validation_h3         = tf.nn.sigmoid(tf.matmul(validation_h2, weights['h3']) + biases['b3'])
        validation_h4         = tf.nn.sigmoid(tf.matmul(validation_h3, weights['h4']) + biases['b4'])
        validation_prediction = tf.nn.softmax(tf.matmul(validation_h4, weights['out']) + biases['out'])

    with tf.name_scope("testing"): 
        testing_h1            = tf.nn.sigmoid(tf.matmul(testData, weights['h1']) + biases['b1'])
        testing_h2            = tf.nn.sigmoid(tf.matmul(testing_h1, weights['h2']) + biases['b2'])
        testing_h3            = tf.nn.sigmoid(tf.matmul(testing_h2, weights['h3']) + biases['b3'])
        testing_h4            = tf.nn.sigmoid(tf.matmul(testing_h3, weights['h4']) + biases['b4'])
        testing_prediction    = tf.nn.softmax(tf.matmul(testing_h4, weights['out']) + biases['out'])              
        testing_pred_onehot   = tf.one_hot(tf.argmax(testing_prediction, dimension = 1), depth = 2)
    
    saver = tf.train.Saver()

init = tf.global_variables_initializer()


Exception ignored in: <generator object get_controller at 0x1a1c275e08>
Traceback (most recent call last):
  File "/Users/ybercovich/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/ops.py", line 4345, in get_controller
    type(default))
AssertionError: Nesting violated for default stack of <class 'tensorflow.python.framework.ops.Graph'> objects


Instructions for updating:
Use the `axis` argument instead


In [ ]:
regul_vals = [0, 1.0, 0.1, 0.01]
keep_probs = [1.0, 0.99, 0.75, 0.50]
#maxnorms   = [None, 3, 4]
maxnorms   = [3]
learning_rates = [1.0,0.1,0.05]

accuracy_list = []


# Start training
for lr in learning_rates:
    for mn in maxnorms:
        for keepProb in keep_probs:
            for regul in regul_vals:
                with tf.Session(graph=graph) as sess:

                    # Run the initializer
                    tf.global_variables_initializer().run()
                    print("Initialized")
                    merged = tf.summary.merge_all()
                    stat_writer = tf.summary.FileWriter( '/tmp/adult_census/%d_%f_%d_%f_%f/'%(int(time.time()), lr, mn, keepProb,regul), sess.graph)
                    run_metadata = tf.RunMetadata()

                    for step in range(1, num_steps+1):
                        batchIndex = np.random.choice(range(trainData.shape[0]), replace=False, size=batch_size)
                        batch_x  = trainData.iloc[ batchIndex, : ].as_matrix()
                        batch_y  = trainLabel[ batchIndex ]

                        feed_dict = {X : batch_x, Y : batch_y, initial_learning_rate: lr , beta_regul : regul, keep_prob: keepProb }

                        summary, _, l, predictions = sess.run([merged, optimizer, loss, train_prediction], feed_dict=feed_dict, run_metadata=run_metadata)

                        if step % display_step == 0 or step == 1:
                            print("Minibatch loss at step %d: %f" % (step, l))
                            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_y))
                            print("Validation accuracy: %.1f%%" % accuracy(validation_prediction.eval(), validationLabel))
                            print("Test accuracy: %.1f%%" % accuracy(testing_prediction.eval(), testLabel))
                            #calcTestMetrics(testing_prediction.eval(), test_y)
                            #print (sklearn.metrics.classification_report(digits=2, y_true=test_Y, y_pred=testing_prediction_onehot.eval()))
                            saver.save(global_step=global_step, sess=sess, save_path='/tmp/adult_model/')
                            print ("saved")
                            run_metadata = tf.RunMetadata()
                            stat_writer.add_run_metadata(run_metadata, 'step%03d' % step)
                            stat_writer.add_summary(summary, step)
                            print('Adding run metadata for', step)


                            print ("")

                    accuracy_list.append( {'learning_rate': lr, 'keep_prob': keepProb, 'l2reg': regul,  'acc': accuracy(testing_prediction.eval(), testLabel) })
                    print("Optimization Finished!")
                    stat_writer.close()
                    


Initialized
Minibatch loss at step 1: 2.364786
Minibatch accuracy: 31.2%
Validation accuracy: 58.8%
Test accuracy: 61.9%
saved
Adding run metadata for 1

Minibatch loss at step 100: 1.784018
Minibatch accuracy: 72.7%
Validation accuracy: 24.3%
Test accuracy: 23.6%
saved
Adding run metadata for 100

Minibatch loss at step 200: 2.196585
Minibatch accuracy: 76.6%
Validation accuracy: 75.7%
Test accuracy: 76.4%
saved
Adding run metadata for 200

Minibatch loss at step 300: 2.994788
Minibatch accuracy: 77.3%
Validation accuracy: 75.7%
Test accuracy: 76.4%
saved
Adding run metadata for 300

Minibatch loss at step 400: 2.236602
Minibatch accuracy: 78.1%
Validation accuracy: 75.7%
Test accuracy: 76.4%
saved
Adding run metadata for 400

Minibatch loss at step 500: 1.192016
Minibatch accuracy: 75.8%
Validation accuracy: 75.7%
Test accuracy: 76.4%
saved
Adding run metadata for 500

Minibatch loss at step 600: 0.601973
Minibatch accuracy: 77.3%
Validation accuracy: 75.7%
Test accuracy: 76.4%
saved

saved
Adding run metadata for 5500

Minibatch loss at step 5600: 2.804009
Minibatch accuracy: 73.4%
Validation accuracy: 75.7%
Test accuracy: 76.4%
saved
Adding run metadata for 5600

Minibatch loss at step 5700: 1.032332
Minibatch accuracy: 78.1%
Validation accuracy: 75.7%
Test accuracy: 76.4%
saved
Adding run metadata for 5700

Minibatch loss at step 5800: 1.247030
Minibatch accuracy: 85.2%
Validation accuracy: 75.7%
Test accuracy: 76.4%
saved
Adding run metadata for 5800

Minibatch loss at step 5900: 0.657808
Minibatch accuracy: 75.0%
Validation accuracy: 75.7%
Test accuracy: 76.4%
saved
Adding run metadata for 5900

Minibatch loss at step 6000: 2.999038
Minibatch accuracy: 76.6%
Validation accuracy: 75.7%
Test accuracy: 76.4%
saved
Adding run metadata for 6000

Minibatch loss at step 6100: 1.917034
Minibatch accuracy: 82.0%
Validation accuracy: 75.7%
Test accuracy: 76.4%
saved
Adding run metadata for 6100

Minibatch loss at step 6200: 1.458774
Minibatch accuracy: 76.6%
Validation a

Minibatch loss at step 1000: 4669.381638
Minibatch accuracy: 71.1%
Validation accuracy: 75.7%
Test accuracy: 76.4%
saved
Adding run metadata for 1000

Minibatch loss at step 1100: 4669.268294
Minibatch accuracy: 78.1%
Validation accuracy: 75.7%
Test accuracy: 76.4%
saved
Adding run metadata for 1100

Minibatch loss at step 1200: 4669.382774
Minibatch accuracy: 72.7%
Validation accuracy: 75.7%
Test accuracy: 76.4%
saved
Adding run metadata for 1200

Minibatch loss at step 1300: 4669.421215
Minibatch accuracy: 71.9%
Validation accuracy: 75.7%
Test accuracy: 76.4%
saved
Adding run metadata for 1300

Minibatch loss at step 1400: 4669.344524
Minibatch accuracy: 71.9%
Validation accuracy: 75.7%
Test accuracy: 76.4%
saved
Adding run metadata for 1400

Minibatch loss at step 1500: 4669.541579
Minibatch accuracy: 73.4%
Validation accuracy: 75.7%
Test accuracy: 76.4%
saved
Adding run metadata for 1500

Minibatch loss at step 1600: 4669.461056
Minibatch accuracy: 75.0%
Validation accuracy: 75.7%


saved
Adding run metadata for 6400

Minibatch loss at step 6500: 4669.351354
Minibatch accuracy: 71.1%
Validation accuracy: 75.7%
Test accuracy: 76.4%
saved
Adding run metadata for 6500

Minibatch loss at step 6600: 4669.285132
Minibatch accuracy: 77.3%
Validation accuracy: 75.7%
Test accuracy: 76.4%
saved
Adding run metadata for 6600

Minibatch loss at step 6700: 4669.331324
Minibatch accuracy: 77.3%
Validation accuracy: 75.7%
Test accuracy: 76.4%
saved
Adding run metadata for 6700

Minibatch loss at step 6800: 4669.365900
Minibatch accuracy: 73.4%
Validation accuracy: 75.7%
Test accuracy: 76.4%
saved
Adding run metadata for 6800

Minibatch loss at step 6900: 4669.354321
Minibatch accuracy: 71.1%
Validation accuracy: 75.7%
Test accuracy: 76.4%
saved
Adding run metadata for 6900

Minibatch loss at step 7000: 4669.281829
Minibatch accuracy: 78.1%
Validation accuracy: 75.7%
Test accuracy: 76.4%
saved
Adding run metadata for 7000

Minibatch loss at step 7100: 4669.321436
Minibatch accurac

saved
Adding run metadata for 1800

Minibatch loss at step 1900: 467.381867
Minibatch accuracy: 82.8%
Validation accuracy: 75.7%
Test accuracy: 76.4%
saved
Adding run metadata for 1900

Minibatch loss at step 2000: 467.474775
Minibatch accuracy: 76.6%
Validation accuracy: 75.7%
Test accuracy: 76.4%
saved
Adding run metadata for 2000

Minibatch loss at step 2100: 467.363185
Minibatch accuracy: 82.0%
Validation accuracy: 75.7%
Test accuracy: 76.4%
saved
Adding run metadata for 2100

Minibatch loss at step 2200: 467.476880
Minibatch accuracy: 71.9%
Validation accuracy: 75.7%
Test accuracy: 76.4%
saved
Adding run metadata for 2200

Minibatch loss at step 2300: 467.458079
Minibatch accuracy: 73.4%
Validation accuracy: 75.7%
Test accuracy: 76.4%
saved
Adding run metadata for 2300

Minibatch loss at step 2400: 467.410179
Minibatch accuracy: 77.3%
Validation accuracy: 75.7%
Test accuracy: 76.4%
saved
Adding run metadata for 2400

Minibatch loss at step 2500: 467.459675
Minibatch accuracy: 73.4

saved
Adding run metadata for 7300

Minibatch loss at step 7400: 467.447106
Minibatch accuracy: 74.2%
Validation accuracy: 75.7%
Test accuracy: 76.4%
saved
Adding run metadata for 7400

Minibatch loss at step 7500: 467.386998
Minibatch accuracy: 79.7%
Validation accuracy: 75.7%
Test accuracy: 76.4%
saved
Adding run metadata for 7500

Minibatch loss at step 7600: 467.409342
Minibatch accuracy: 78.1%
Validation accuracy: 75.7%
Test accuracy: 76.4%
saved
Adding run metadata for 7600

Minibatch loss at step 7700: 467.425440
Minibatch accuracy: 76.6%
Validation accuracy: 75.7%
Test accuracy: 76.4%
saved
Adding run metadata for 7700

Minibatch loss at step 7800: 467.436624
Minibatch accuracy: 75.8%
Validation accuracy: 75.7%
Test accuracy: 76.4%
saved
Adding run metadata for 7800

Minibatch loss at step 7900: 467.461906
Minibatch accuracy: 72.7%
Validation accuracy: 75.7%
Test accuracy: 76.4%
saved
Adding run metadata for 7900

Minibatch loss at step 8000: 467.436875
Minibatch accuracy: 76.6

saved
Adding run metadata for 2700

Minibatch loss at step 2800: 47.268554
Minibatch accuracy: 74.2%
Validation accuracy: 75.7%
Test accuracy: 76.4%
saved
Adding run metadata for 2800

Minibatch loss at step 2900: 47.204412
Minibatch accuracy: 78.1%
Validation accuracy: 75.7%
Test accuracy: 76.4%
saved
Adding run metadata for 2900

Minibatch loss at step 3000: 47.252022
Minibatch accuracy: 75.8%
Validation accuracy: 75.7%
Test accuracy: 76.4%
saved
Adding run metadata for 3000

Minibatch loss at step 3100: 47.231273
Minibatch accuracy: 77.3%
Validation accuracy: 75.7%
Test accuracy: 76.4%
saved
Adding run metadata for 3100

Minibatch loss at step 3200: 47.222833
Minibatch accuracy: 77.3%
Validation accuracy: 75.7%
Test accuracy: 76.4%
saved
Adding run metadata for 3200

Minibatch loss at step 3300: 47.226696
Minibatch accuracy: 78.1%
Validation accuracy: 75.7%
Test accuracy: 76.4%
saved
Adding run metadata for 3300

Minibatch loss at step 3400: 47.381674
Minibatch accuracy: 73.4%
Valid

saved
Adding run metadata for 8200

Minibatch loss at step 8300: 47.158197
Minibatch accuracy: 82.8%
Validation accuracy: 75.7%
Test accuracy: 76.4%
saved
Adding run metadata for 8300

Minibatch loss at step 8400: 47.262559
Minibatch accuracy: 74.2%
Validation accuracy: 75.7%
Test accuracy: 76.4%
saved
Adding run metadata for 8400

Minibatch loss at step 8500: 47.225848
Minibatch accuracy: 78.1%
Validation accuracy: 75.7%
Test accuracy: 76.4%
saved
Adding run metadata for 8500

Minibatch loss at step 8600: 47.513904
Minibatch accuracy: 74.2%
Validation accuracy: 75.7%
Test accuracy: 76.4%
saved
Adding run metadata for 8600

Minibatch loss at step 8700: 47.248760
Minibatch accuracy: 78.1%
Validation accuracy: 75.7%
Test accuracy: 76.4%
saved
Adding run metadata for 8700

Minibatch loss at step 8800: 47.263643
Minibatch accuracy: 73.4%
Validation accuracy: 75.7%
Test accuracy: 76.4%
saved
Adding run metadata for 8800

Minibatch loss at step 8900: 47.343687
Minibatch accuracy: 63.3%
Valid

In [ ]:
for i in accuracy_list:
    print (i)